# Recording Data Provenance with ``aiida-gromacs``

This tutorial follows the first six steps of Justin Lemkul’s [lysozyme tutorial](http://www.mdtutorials.com/gmx/lysozyme/). We will not explain each individual step as this can be found on Justin’s webpage, but we will link to each page and show the ``aiida-gromacs`` equivalant command.

Please also note the slight differences in commands between the tutorial and that by Justin Lemkul is simply down to the way we are recording provenance, which requires non-interactive input into the gromacs tools.

Also at each of the below steps you should run ``verdi`` (the command line interface utility for AiiDA) to view the status of the submitted process before moving onto the next step, you do this by:

In [22]:
! verdi process list -a

PK    Created    Process label    Process State    Process status
----  ---------  ---------------  ---------------  ----------------

Total results: 0

Report: last time an entry changed state: 12m ago (at 07:52:25 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


In this notebook AiiDA, the aiida-gromacs plugin and dependent tools are pre-installed. Here's a brief description of the tools used; 
* AiiDA uses a [PostgreSQL](https://www.postgresql.org) database to store all data produced and the links between input and output files for each command run. Each submitted command is termed a process in AiiDA. 
* Communication between submitted processes are handled with [RabbitMQ](https://www.rabbitmq.com/) and submitted processes are handled with a deamon process that runs in the background. 
* ``aiida-gromacs`` requires an installation of [GROMACS](https://www.gromacs.org/) and the path to where it is installed.

1. We will start from the [pbd2gmx](http://www.mdtutorials.com/gmx/lysozyme/01_pdb2gmx.html)  step of Justin’s tutorial, with the ``aiida-gromacs`` equivalent python wrapper ``gmx_pdb2gmx``:

In [23]:
! gmx_pdb2gmx -f gromacs_files/1AKI_clean.pdb -ff oplsaa -water spce -o 1AKI_forcefield.gro -p 1AKI_topology.top -i 1AKI_restraints.itp

A successfully finished process will exit with code [0]. 

In [24]:
! verdi process list -a

  PK  Created    Process label       Process State    Process status
----  ---------  ------------------  ---------------  ----------------
 663  6s ago     Pdb2gmxCalculation  ⏹ Finished [0]

Total results: 1

Report: last time an entry changed state: 5s ago (at 08:04:38 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


2. Next we will create the [box and then solvate](http://www.mdtutorials.com/gmx/lysozyme/03_solvate.html):

In [25]:
! gmx_editconf -f 1AKI_forcefield.gro -center 0 -d 1.0 -bt cubic -o 1AKI_newbox.gro

In [26]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  21s ago    Pdb2gmxCalculation   ⏹ Finished [0]
 671  6s ago     EditconfCalculation  ⏹ Finished [0]

Total results: 2

Report: last time an entry changed state: 5s ago (at 08:04:53 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


In [27]:
! gmx_solvate -cp 1AKI_newbox.gro -cs spc216.gro -p 1AKI_topology.top -o 1AKI_solvated.gro

In [28]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  36s ago    Pdb2gmxCalculation   ⏹ Finished [0]
 671  21s ago    EditconfCalculation  ⏹ Finished [0]
 677  3s ago     SolvateCalculation   ⏹ Finished [0]

Total results: 3

Report: last time an entry changed state: 3s ago (at 08:05:11 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


3. & 4. Then add [ions](http://www.mdtutorials.com/gmx/lysozyme/04_ions.html) to neutralise the system after preprocessing the topology:

In [29]:
! gmx_grompp -f gromacs_files/ions.mdp -c 1AKI_solvated.gro -p 1AKI_topology.top -o 1AKI_ions.tpr

In [30]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  46s ago    Pdb2gmxCalculation   ⏹ Finished [0]
 671  31s ago    EditconfCalculation  ⏹ Finished [0]
 677  13s ago    SolvateCalculation   ⏹ Finished [0]
 685  3s ago     GromppCalculation    ⏹ Finished [0]

Total results: 4

Report: last time an entry changed state: 2s ago (at 08:05:21 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


In [31]:
! gmx_genion -s 1AKI_ions.tpr -p 1AKI_topology.top -pname NA -nname CL -neutral true -o 1AKI_solvated_ions.gro

In [32]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  1m ago     Pdb2gmxCalculation   ⏹ Finished [0]
 671  52s ago    EditconfCalculation  ⏹ Finished [0]
 677  34s ago    SolvateCalculation   ⏹ Finished [0]
 685  24s ago    GromppCalculation    ⏹ Finished [0]
 691  5s ago     GenionCalculation    ⏹ Finished [0]

Total results: 5

Report: last time an entry changed state: 4s ago (at 08:05:40 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


5. & 6. Then [minimise](http://www.mdtutorials.com/gmx/lysozyme/05_EM.html) the system after preprocessing the topology:

In [33]:
! gmx_grompp -f gromacs_files/min.mdp -c 1AKI_solvated_ions.gro -p 1AKI_topology.top -o 1AKI_minimised.tpr

In [34]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  1m ago     Pdb2gmxCalculation   ⏹ Finished [0]
 671  1m ago     EditconfCalculation  ⏹ Finished [0]
 677  52s ago    SolvateCalculation   ⏹ Finished [0]
 685  42s ago    GromppCalculation    ⏹ Finished [0]
 691  23s ago    GenionCalculation    ⏹ Finished [0]
 699  5s ago     GromppCalculation    ⏹ Finished [0]

Total results: 6

Report: last time an entry changed state: 4s ago (at 08:05:58 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


In [35]:
! gmx_mdrun -s 1AKI_minimised.tpr -c 1AKI_minimised.gro -e 1AKI_minimised.edr -g 1AKI_minimised.log -o 1AKI_minimised.trr

In [38]:
! verdi process list -a

  PK  Created    Process label        Process State    Process status
----  ---------  -------------------  ---------------  ----------------
 663  1m ago     Pdb2gmxCalculation   ⏹ Finished [0]
 671  1m ago     EditconfCalculation  ⏹ Finished [0]
 677  1m ago     SolvateCalculation   ⏹ Finished [0]
 685  1m ago     GromppCalculation    ⏹ Finished [0]
 691  48s ago    GenionCalculation    ⏹ Finished [0]
 699  30s ago    GromppCalculation    ⏹ Finished [0]
 705  21s ago    MdrunCalculation     ⏹ Finished [0]

Total results: 7

Report: last time an entry changed state: 6s ago (at 08:06:21 on 2023-10-11)
Report: Checking daemon load... OK
Report: Using 0% of the available daemon worker slots.


We can view the provenance graph of these processes, which shows how inputs and outputs of each process are connected to other processes. To save the provenance graph of all finished processes, replace the primary key value <PK> in the command below with that of the most recently run process.

In [39]:
! verdi node graph generate 705

Success: Output written to `705.dot.pdf`


At the end of a project, the AiiDA database can be saved as an AiiDA archive file (sqlite/zip format) for long term storage and to share your data and provenance with others. 

In [40]:
! verdi archive create --all lysozyme_tutorial.aiida

Report: 
Archive Parameters
--------------------  -----------------------
Path                  lysozyme_tutorial.aiida
Version               main_0001
Compression           6

Inclusion rules
----------------------------  -----
Computers/Nodes/Groups/Users  All
Computer Authinfos            False
Node Comments                 True
Node Logs                     True

Report: Validating Nodes                                           
Report: Creating archive with:
---------  ---
users        1
computers    2
nodes      139
links       62
---------  ---
Report: Finalizing archive creation...                             
Report: Archive created successfully
Success: wrote the export archive file to lysozyme_tutorial.aiida


We hope to share further tutorials on loading, querying and displaying data from AiiDA archives. Watch this space!